# **Introduction**
<br>
Unicorn is a family business owned by 2 stakeholders who are very invested in their business.<br>
The company focuses on e-commerce, trusting the common idea that the online sector has been slowly eating up market share in the past two decades. <br>
Unicorn's platform allows people to buy products online: from books, toys, clothes, and shoes to food, furniture, and other household items.<br>
<br>
We have been asked to analyze Unicorn's data, find interesting insights, and identify weak areas and opportunities for Unicorn to boost its business growth.



# **Connecting to the database**

In [2]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
# import secrets # secrets.py contain db connexion information and isn't provided here

In [ ]:
# For safety reasons the address to the db is encrypted

engine = create_engine(f'postgresql+psycopg2://{secrets["ID"]}:{secrets["PASSWORD"]}@{secrets["ADDRESS"]}:{secrets["PORT"]}/{secrets["NAME"]}')

In [ ]:
connection = engine.connect()

Due to integrity and safety issues, we cannot provide a connection to the database.
Thus, here will only be displayed code for EDA and results.

# **Cleaning**
Checking consistency of data using python

In [ ]:
product = pd.read_sql('SELECT * FROM product', connection)
orders = pd.read_sql('SELECT * FROM orders', connection)
order_details = pd.read_sql('SELECT * FROM order_details', connection)
customers = pd.read_sql('SELECT * FROM customers', connection)

In [ ]:
# Checking for missing values, only found one issue:
# Missing postal code for the city of Burlington
orders[orders['shipping_postal_code'].isna()]

,order_id,customer_id,order_date,shipping_city,shipping_state,shipping_region,shipping_country,shipping_postal_code,shipping_date,shipping_mode
1029,CA-2016-117086,616,2016-11-08,Burlington,Vermont,East,United States,NaN,2016-11-12,Standard Class
1624,CA-2016-162887,718,2016-11-07,Burlington,Vermont,East,United States,NaN,2016-11-09,Second Class
2896,CA-2018-104066,616,2018-12-05,Burlington,Vermont,East,United States,NaN,2018-12-10,Standard Class
4657,US-2017-150140,777,2017-04-06,Burlington,Vermont,East,United States,NaN,2017-04-10,Standard Class
4714,US-2017-165505,168,2017-01-23,Burlington,Vermont,East,United States,NaN,2017-01-27,Standard Class
4852,US-2018-127292,624,2018-01-19,Burlington,Vermont,East,United States,NaN,2018-01-23,Standard Class


Overall the missing values for the city of Burlington seem to be the only notable issue.
However, since we have no use of the postal code and it doesn't impact our analysis we decide to leave the data untouched.

# **EDA**

In [ ]:
# Number of customers and number of order per customers

query = """

SELECT 	COUNT(DISTINCT customer_id) customer_count,
				COUNT(order_id) order_count,
				COUNT(order_id) / COUNT(DISTINCT customer_id) avg_order_per_customer
FROM orders

"""

pd.read_sql(query, connection)

,customer_count,order_count,avg_order_per_customer
0,793,5013,6


In [ ]:
# Total spent by customers

query = """
SELECT	c.customer_id,
				SUM(od.order_sales) total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
GROUP BY c.customer_id
ORDER BY total_spent

"""

pd.read_sql(query, connection)

# We see that there are quite noticeable differences between customers' spendings
# Could that be due to customer_segment?

,customer_id,total_spent
0,456,5.0
1,738,5.0
2,546,16.0
3,124,17.0
4,657,22.0
...,...,...
788,758,14596.0
789,623,15117.0
790,388,16744.0
791,731,19050.0


In [ ]:
# Distribution of customers by type and their associated spending

query = """

SELECT	c.customer_segment,
				COUNT(c.customer_segment) customer_counts,
				SUM(od.order_sales) total_spent,
        SUM(od.order_sales) / COUNT(c.customer_segment) avg_spent_per_customer
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
GROUP BY c.customer_segment
ORDER BY total_spent

"""

pd.read_sql(query, connection)

# Despite having some customer types dominating in total spending,
# it seems that customer_segments have proportionaly equal spending... hence outliers exist in every categories.
# What could explain the behavior of customers who spend a lot or not that much?
# Unfortunately, it seems that we cannot easily infer customer's behaviours from the current data.

,customer_segment,customer_counts,total_spent,avg_spent_per_customer
0,Home Office,1781,431012.0,242.005615
1,Corporate,3021,707561.0,234.214167
2,Consumer,5195,1168676.0,224.961694


In [ ]:
# Distribution of customers by type and their associated spending

query = """

SELECT	c.customer_segment,
				COUNT(c.customer_segment) customer_counts,
				SUM(od.order_sales) total_spent,
        SUM(od.order_sales) / COUNT(c.customer_segment) avg_spent_per_customer
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
GROUP BY c.customer_segment
ORDER BY total_spent

"""

pd.read_sql(query, connection)

# Despite having some customer types dominating in total spending,
# it seems that customer_segments have proportionaly equal spending... hence outliers exist in every categories.
# What could explain the behavior of customers who spend a lot or not that much?
# Unfortunately, it seems that we cannot easily infer customer's behaviours from the current data.

In [ ]:
# Most profitable states

query = """

SELECT	o.shipping_state,
				SUM(od.order_profits) profits
FROM orders o
JOIN order_details od
ON o.order_id = od.order_id
GROUP BY o.shipping_state
ORDER BY profits DESC

"""

pd.read_sql(query, connection)

# Where we notice that some states are concerningly in deficit.
# What could be the reasons for these states to underperform?

,shipping_state,profits
0,New York,78557.0
1,California,76368.0
2,Washington,33390.0
3,Michigan,24458.0
4,Virginia,18434.0
5,Indiana,18382.0
6,Georgia,16247.0
7,Kentucky,11140.0
8,Minnesota,10828.0
9,Delaware,9979.0


In [ ]:
# But first, let's also look at the average profit per year

query = """

SELECT	EXTRACT(YEAR FROM o.shipping_date) years,
				SUM(od.order_profits)
FROM orders o
JOIN order_details od
ON o.order_id = od.order_id
GROUP BY EXTRACT(YEAR FROM o.shipping_date)

"""

pd.read_sql(query, connection)

# While Unicorn benefits seem to steadily increase throughout the years,
# we notice a dramatic drop for the year 2019.
# Although it can be explained by the available data not covering the whole 2019 year, and stopping of the 5th of January 2019.
# Hence, overall Unicorn's profit are steadily increasing.

,years,sum
0,2016.0,62807.0
1,2017.0,82523.0
2,2018.0,91849.0
3,2019.0,937.0
4,2015.0,51840.0


In [ ]:
# Let's also have a look at the most profitable product categories

query = """


SELECT	p.product_category,
				SUM(od.order_profits) profits
FROM order_details od
JOIN product p
ON od.product_id = p.product_id
GROUP BY p.product_category
ORDER BY profits DESC


"""

pd.read_sql(query, connection)

# While Technology and Office Supplies category appear to be the main source of revenue for Unicorn,
# it could be interesting to look deeper in the Furniture category, to raise it as a potential market

,product_category,profits
0,Technology,146737.0
1,Office Supplies,124364.0
2,Furniture,18855.0


In [ ]:
# Use of discount per user / versus total spent:
# Who are the users that makes us at loss?

query = """

SELECT	c.customer_id,
				SUM(od.order_profits) profits,
				SUM(order_discount) discount
FROM orders o
JOIN order_details od
ON o.order_id = od.order_id
JOIN customers c
ON o.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY profits ASC


"""

pd.read_sql(query, connection)

# It is hard to read, but on first view we can suspect that users that generate deficit
# are usually using more discounts than user who generate profits

,customer_id,profits,discount
0,166,-6625.0,1.8
1,308,-4109.0,1.5
2,478,-3583.0,5.1
3,694,-3336.0,3.3
4,330,-2797.0,2.9
...,...,...,...
788,7,5445.0,4.8
789,335,5623.0,0.2
790,672,5756.0,1.4
791,623,6976.0,1.7


In [ ]:
# Zooming out of users, let's test our assumption at the State level:
# What is the discount effect on profit? for state that make no profit
# and what is the average discount for state that make profit

query = """

WITH tab1 AS (
SELECT	o.shipping_state,
				SUM(od.order_profits) AS profit,
        COUNT(od.order_id) AS num_of_orders,
			  AVG(od.order_discount) average_discount,
        AVG(od.order_profit_ratio) AS profit_ratio
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id

GROUP BY o.shipping_state
ORDER BY o.shipping_state
  )

SELECT	shipping_state,
        profit,
        num_of_orders,
        profit_ratio,
        average_discount
FROM tab1
WHERE average_discount > 0
ORDER BY profit

"""

pd.read_sql(query, connection)

# Here we can see that the average discount is unusually hight for states that make no profit
# States that make profit offer less discount than states that lose profit.
# There seem to exist a negative correlation between profit/discount.
# Could we drill down these result and check if inside these states there are cities that are outliers, or if they compose an even substrate?

,shipping_state,profit,num_of_orders,profit_ratio,average_discount
0,Texas,-25714.0,985,-0.341888,0.370193
1,Ohio,-16950.0,468,-0.073355,0.325000
2,Pennsylvania,-15550.0,587,-0.085605,0.328620
3,Illinois,-12607.0,492,-0.391565,0.390041
4,North Carolina,-7572.0,248,0.007500,0.283871
5,Colorado,-6541.0,181,-0.124917,0.317127
6,Tennessee,-5347.0,183,-0.016448,0.291257
7,Arizona,-3432.0,224,-0.066027,0.303571
8,Florida,-3412.0,383,-0.017572,0.299347
9,Oregon,-1187.0,124,-0.049677,0.288710


In [ ]:
# Let's have a look on how the discount rate holds throughout the years:

query = """

WITH tab1 AS (
SELECT	o.shipping_state,
  			EXTRACT(year FROM o.shipping_date) AS years,
				SUM(od.order_profits) AS profit,
        COUNT(od.order_id) AS num_of_orders,
			  AVG(od.order_discount) average_discount,
        AVG(od.order_profit_ratio) AS profit_ratio
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id

GROUP BY years, o.shipping_state
ORDER BY o.shipping_state, years
  ),
state_discount_diff AS (
SELECT	shipping_state,
  			years,
        profit,
        num_of_orders,
        profit_ratio,
        average_discount
FROM tab1
WHERE average_discount > 0
  ),

profit_below AS (
SELECT	years,
        AVG(average_discount) AS avg_discount_state_negative
FROM state_discount_diff
WHERE profit < 0
GROUP BY years
),
profit_above AS (
  SELECT	years,
        AVG(average_discount) AS avg_discount_state_positive
FROM state_discount_diff
WHERE profit > 0
GROUP BY years
)
SELECT	pb.years,
				pb.avg_discount_state_negative,
        pa.avg_discount_state_positive
FROM profit_below pb
JOIN profit_above pa ON pb.years = pa.years
ORDER BY pb.years

"""

pd.read_sql(query, connection)

# We notice that the average discount is consistent throughout the years.
# It is concerning, since we noticed a negative profit/discount correlation.
# This behavior should be addressed, as the trend impacts Unicorn's performances.

,years,avg_discount_state_negative,avg_discount_state_positive
0,2015.0,0.314312,0.087436
1,2016.0,0.321316,0.045415
2,2017.0,0.337206,0.088250
3,2018.0,0.326655,0.067358
4,2019.0,0.353750,0.109524


In [ ]:
# Let's drill down to rank the cities that makes the least profit per state.
# There are 10 states which have an average negative profit.
# Thus we look for the cities' profit/deficit within these states.

query = """

WITH state_below_profit AS(
SELECT	o.shipping_state
FROM order_details od
JOIN orders o ON od.order_id = o.order_id
GROUP BY o.shipping_state
HAVING AVG(od.order_profits) < 0
),
state_below_profit2 AS (
SELECT	sbp.shipping_state,
				o.shipping_city,
        SUM(od.order_profits) total_profit_deficit
FROM state_below_profit sbp
JOIN orders o ON sbp.shipping_state = o.shipping_state
JOIN order_details od ON od.order_id = o.order_id
GROUP BY sbp.shipping_state, o.shipping_city
ORDER BY sbp.shipping_state
)

SELECT	shipping_state,
				shipping_city,
        total_profit_deficit,
        AVG(total_profit_deficit) OVER(PARTITION BY shipping_state) AS average_state_profit
FROM state_below_profit2
ORDER BY shipping_state, total_profit_deficit

"""

pd.read_sql(query, connection)

# We see a tendency: the largest cities within these states are usually the ones that perform poorly.
# These could be possibly considered outliers that distort the overall performance of these states.
# We could look at the discount offered within these cities to see if it is this parameter that mainly affects the performances.

,shipping_state,shipping_city,total_profit_deficit,average_state_profit
0,Arizona,Phoenix,-2793.0,-264.000000
1,Arizona,Mesa,-558.0,-264.000000
2,Arizona,Yuma,-466.0,-264.000000
3,Arizona,Scottsdale,-90.0,-264.000000
4,Arizona,Tucson,-66.0,-264.000000
...,...,...,...,...
222,Texas,Beaumont,59.0,-476.185185
223,Texas,Port Arthur,106.0,-476.185185
224,Texas,Lubbock,147.0,-476.185185
225,Texas,Brownsville,203.0,-476.185185


In [ ]:
# Let's compare the discounts within states that are in deficit
# between cities that make profit and those that are not

query = """

WITH state_below_profit AS(
SELECT	o.shipping_state
FROM order_details od
JOIN orders o ON od.order_id = o.order_id
GROUP BY o.shipping_state
HAVING AVG(od.order_profits) < 0
),
state_below_profit2 AS (
SELECT	sbp.shipping_state,
				o.shipping_city,
        SUM(od.order_profits) total_profit_deficit,
  			AVG(od.order_discount) order_discount_avg
FROM state_below_profit sbp
JOIN orders o ON sbp.shipping_state = o.shipping_state
JOIN order_details od ON od.order_id = o.order_id
GROUP BY sbp.shipping_state, o.shipping_city
ORDER BY sbp.shipping_state
),

tab1 AS(
SELECT	shipping_state,
				shipping_city,
        total_profit_deficit,
        AVG(total_profit_deficit) OVER(PARTITION BY shipping_state) AS state_average_profit,
        ROUND(order_discount_avg::numeric, 2) average_discount
FROM state_below_profit2
ORDER BY shipping_state, total_profit_deficit
),

good AS(
SELECT shipping_state,
AVG(CASE
			WHEN total_profit_deficit >=0 THEN average_discount END) AS avg_discount_for_profit_cities
FROM tab1
GROUP BY shipping_state
),
bad AS(
SELECT shipping_state,
AVG(CASE
			WHEN total_profit_deficit < 0 THEN average_discount END) AS avg_discount_for_deficit_cities
FROM tab1
GROUP BY shipping_state
)

SELECT	g.shipping_state,
				g.avg_discount_for_profit_cities,
				b.avg_discount_for_deficit_cities,
        b.avg_discount_for_deficit_cities - g.avg_discount_for_profit_cities AS discount_avg_diff
FROM good g
JOIN bad b on g.shipping_state = b.shipping_state

"""

pd.read_sql(query, connection)

# Even though the difference isn't as clear as between states that makes profit or not
# we observe that generally the cities that make profit within these states have lower discount
# than the cities that are in deficit.

,shipping_state,avg_discount_for_profit_cities,avg_discount_for_deficit_cities,discount_avg_diff
0,Tennessee,0.290000,0.308889,0.018889
1,Oregon,0.292000,0.295000,0.003000
2,Colorado,0.247143,0.384444,0.137302
3,Ohio,0.357500,0.356500,-0.001000
4,Arizona,0.224286,0.341667,0.117381
5,North Carolina,0.252727,0.335000,0.082273
6,Florida,0.252609,0.390000,0.137391
7,Pennsylvania,0.330000,0.358571,0.028571
8,Illinois,0.256250,0.454000,0.197750
9,Texas,0.276667,0.424167,0.147500


# **Conclusion and observations**

Through our EDA we uncover concerning patterns within Unicorn's data that hinder the company revenues.<br>
A negative correlation profit/discount has been noticed.<br>
Although we notice this pattern, we cannot infer causations.<br>
Thus, while asking for more data to understand better these patterns and our customers behaviours, we also propose a hands-on solution to help balancing discounts in states, cities, product categories and products that generate deficit.<br>
<br>
This solution can be reviewed in our Tableau-produced dashboard:<br>:
https://public.tableau.com/app/profile/bertrand.flanet/viz/Unicorn_viz/DashboardUnicorn2?publish=yes